In [3]:
import tdb
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import cm

In [53]:
data = tdb.load_tdb("hprc_105.tdb/",
                    lfilters=[("chrom", "=", "chr4")]) 
metadata = (pd.read_csv("igsr_samples.tsv", sep='\t')
                .where(lambda x: x["Sample name"].isin(data["sample"].keys()))
                .dropna())

Here I am trying to calculate Fst for all the tandem repeat we have in the databases following the defintion in https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3141729/#APP1title

![alt text](fst.png "Title")

I will calculate w first from the metada and not in the larger natural population for simplicity

In [13]:
counts = metadata["Superpopulation code"].value_counts()
w = counts / counts.sum()

In [26]:
data['allele'].head()

,LocusID,allele_number,allele_length,sequence
0,604481,0,85,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...
1,604481,1,87,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...
2,604481,2,83,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...
3,604481,3,89,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...
4,604481,4,93,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...


In [58]:
samples_per_pop = metadata.groupby('Superpopulation code')['Sample name'].agg(list)



Superpopulation code
AFR    [HG01887, HG02055, HG02145, HG02257, HG02809, ...
AMR    [HG00733, HG00738, HG01243, HG01109, HG01255, ...
EAS    [HG00423, HG00706, HG02074, HG02132, HG00544, ...
SAS    [HG02738, HG02602, HG02683, HG02656, HG04115, ...
Name: Sample name, dtype: object


In [88]:
def hasVariant(df, locus, allele):
    condition = (df['LocusID'] == locus) & (df['allele_number'] == allele)
    return df[condition].shape[0] > 0



In [112]:
data['sample']['HG01887'].head(1)['LocusID']

TypeError: NDFrame.get() missing 1 required positional argument: 'key'

In [121]:
locusID = 604481
allele_number = 3

def calculateFst(locus_row):
    locusID = locus_row['LocusID']
    allele_number = locus_row['allele_number']
    Hs = 0
    Ht_1 = 0
    Ht_2 = 0
    for pop in samples_per_pop.keys():
        number_of_haps = len(samples_per_pop[pop])*2
        has_variants = [hasVariant(data['sample'][s], locusID, allele_number) for s in samples_per_pop[pop]]
        pk = sum(has_variants)/number_of_haps
        qk = 1-pk
        Hs += 2 * w[pop] * pk * qk
        Ht_1 +=  w[pop] * pk 
        Ht_2 +=  w[pop] * qk 
    
    Ht = 2* Ht_1 * Ht_2
    
    Fst = 1 - (Hs/Ht)
    return Fst
    
df = data['allele'].head(10)
df['fst'] = df.apply(calculateFst, axis=1)
df
#calculateFst(data['sample']['HG01887'].head(1))


/var/folders/4f/v5q9cq3n38xfxdz6wmvr39h80000gp/T/ipykernel_5478/1232242501.py:21: RuntimeWarning: invalid value encountered in scalar divide
  Fst = 1 - (Hs/Ht)
/var/folders/4f/v5q9cq3n38xfxdz6wmvr39h80000gp/T/ipykernel_5478/1232242501.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fst'] = df.apply(calculateFst, axis=1)


,LocusID,allele_number,allele_length,sequence,fst
0,604481,0,85,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.013590
1,604481,1,87,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.007131
2,604481,2,83,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.033183
3,604481,3,89,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.020646
4,604481,4,93,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,NaN
5,604481,5,91,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.027007
6,604481,6,91,b'TTACTATGAGATTGCTATCCACTATTTATGTGTGTGTGTGTGTG...,0.013986
7,604481,7,91,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.015597
8,604481,8,87,b'TTACTATGAGATTGCTATCCACTATTTATGTGTGTGTGTGTGTG...,0.013986
9,604481,9,90,b'TTACTATGAGATTGCTATCCACTATTTATATGTGTGTGTGTGTG...,0.015597


In [ ]:
data['fst'] = data['allele'].apply(calculateFst, axis=1)

/var/folders/4f/v5q9cq3n38xfxdz6wmvr39h80000gp/T/ipykernel_5478/1232242501.py:21: RuntimeWarning: invalid value encountered in scalar divide
  Fst = 1 - (Hs/Ht)


LocusID  allele_number
604481   0                  85
         1                  87
         2                  83
         3                  89
         4                  93
                          ... 
664070   198              4318
         199              3599
         200              4186
         201              2060
         202              3107
Name: allele_length, Length: 762908, dtype: uint16